In [1]:
import os
os.getcwd()
os.listdir()
os.listdir("data")

['20251216144313', '날씨 데이터', '법정동별시간별전력사용량.csv', '법정동코드 조회자료.xls']

In [2]:
import sys, xlrd
print(sys.executable)
print(xlrd.__version__)

c:\project-AI\.venv\Scripts\python.exe
2.0.2


In [ ]:
#1-1 법정 데이터 
import pandas as pd

file_path = "data/법정동코드 조회자료.xls"
dong_df = pd.read_excel(file_path)

dong_df.head()

,법정동코드,법정동명,폐지구분
0,1100000000,서울특별시,현존
1,1111000000,서울특별시 종로구,현존
2,1111010100,서울특별시 종로구 청운동,현존
3,1111010200,서울특별시 종로구 신교동,현존
4,1111010300,서울특별시 종로구 궁정동,현존


In [ ]:
# 1-2) 전체코드 분리 (앞 5자리/뒤 5자리)
# 10자리 보장 (앞자리 0 유실 방지)
dong_df["법정동코드"] = dong_df["법정동코드"].astype(str).str.zfill(10)

dong_df["시군구코드"] = dong_df["법정동코드"].str[:5]
dong_df["법정동코드_하위"] = dong_df["법정동코드"].str[5:]

dong_df[["법정동코드","시군구코드","법정동코드_하위"]].head()

,법정동코드,시군구코드,법정동코드_하위
0,1100000000,11000,00000
1,1111000000,11110,00000
2,1111010100,11110,10100
3,1111010200,11110,10200
4,1111010300,11110,10300


In [6]:
# 1-3) 법정동코드(뒤 5자리)가 00000인 행 제거
before = len(dong_df)
dong_df = dong_df[dong_df["법정동코드_하위"] != "00000"].copy()
after = len(dong_df)

print("제거 전:", before, "제거 후:", after, "제거:", before-after)

제거 전: 1112 제거 후: 1086 제거: 26


In [8]:
# 폐지된 법정동 제거
dong_df.columns
if "폐지여부" in dong_df.columns:
    before = len(dong_df)
    dong_df = dong_df[dong_df["폐지여부"] == "존재"].copy()
    after = len(dong_df)
    print("폐지여부 필터 전:", before, "필터 후:", after, "제거:", before-after)
else:
    print("폐지여부 컬럼 없음 -> 스킵")

폐지여부 컬럼 없음 -> 스킵


In [9]:
# 1-4) 법정동 전체명 분리 (시군구명 / 법정동명)
dong_df["법정동명"] = dong_df["법정동명"].astype(str).str.strip()

dong_df["시군구명"] = dong_df["법정동명"].apply(lambda x: " ".join(x.split()[:-1]))
dong_df["법정동명_하위"] = dong_df["법정동명"].apply(lambda x: x.split()[-1])

dong_df[["법정동명","시군구명","법정동명_하위"]].head(10)

,법정동명,시군구명,법정동명_하위
2,서울특별시 종로구 청운동,서울특별시 종로구,청운동
3,서울특별시 종로구 신교동,서울특별시 종로구,신교동
4,서울특별시 종로구 궁정동,서울특별시 종로구,궁정동
5,서울특별시 종로구 효자동,서울특별시 종로구,효자동
6,서울특별시 종로구 창성동,서울특별시 종로구,창성동
7,서울특별시 종로구 통의동,서울특별시 종로구,통의동
8,서울특별시 종로구 적선동,서울특별시 종로구,적선동
9,서울특별시 종로구 통인동,서울특별시 종로구,통인동
10,서울특별시 종로구 누상동,서울특별시 종로구,누상동
11,서울특별시 종로구 누하동,서울특별시 종로구,누하동


In [10]:
# 조인용 테이블 형태로 컬럼 정돈
dong_clean = dong_df[[
    "법정동코드",
    "시군구코드",
    "법정동코드_하위",
    "시군구명",
    "법정동명_하위"
]].drop_duplicates().reset_index(drop=True)

dong_clean.head()

,법정동코드,시군구코드,법정동코드_하위,시군구명,법정동명_하위
0,1111010100,11110,10100,서울특별시 종로구,청운동
1,1111010200,11110,10200,서울특별시 종로구,신교동
2,1111010300,11110,10300,서울특별시 종로구,궁정동
3,1111010400,11110,10400,서울특별시 종로구,효자동
4,1111010500,11110,10500,서울특별시 종로구,창성동


In [11]:
print("dong_clean 행수:", len(dong_clean))
print("법정동코드 중복:", dong_clean["법정동코드"].duplicated().sum())
print("결측치 개수:\n", dong_clean.isna().sum())

dong_clean 행수: 1086
법정동코드 중복: 0
결측치 개수:
 법정동코드       0
시군구코드       0
법정동코드_하위    0
시군구명        0
법정동명_하위     0
dtype: int64


In [21]:
# 저장
dong_clean.to_csv(
    "data/법정동_마스터.csv",
    index=False,
    encoding="utf-8-sig"
)

In [22]:
# 저장 내용 테스트
test = pd.read_csv("data/법정동_마스터.csv")
print(test.shape)
test.head()

(1086, 5)


,법정동코드,시군구코드,법정동코드_하위,시군구명,법정동명_하위
0,1111010100,11110,10100,서울특별시 종로구,청운동
1,1111010200,11110,10200,서울특별시 종로구,신교동
2,1111010300,11110,10300,서울특별시 종로구,궁정동
3,1111010400,11110,10400,서울특별시 종로구,효자동
4,1111010500,11110,10500,서울특별시 종로구,창성동


In [12]:
# 2-1) 전력량 데이터 로드 (인코딩 자동 대응)
import pandas as pd
power_path = "data/법정동별시간별전력사용량.csv"
# 인코딩 이슈 대비: utf-8-sig 우선, 실패하면 cp949
try:
    power_df = pd.read_csv(power_path, encoding="utf-8-sig")
except UnicodeDecodeError:
    power_df = pd.read_csv(power_path, encoding="cp949")

print(power_df.shape)
power_df.head()

(9754804, 5)


,SIGUNGU_CD,BJDONG_CD,USE_YM,USE_HM,FDRCT_VLD_KWH
0,11650,10700,20220628,100,10782.0565
1,11650,10800,20220628,100,11394.8635
2,11650,10900,20220628,100,7273.9620
3,11740,10300,20220628,100,11008.8110
4,11710,11300,20220628,100,2905.1120


In [24]:
os.listdir("data/weather_data")

['SURFACE_ASOS_108_DAY_2022_2022_2023.csv',
 'SURFACE_ASOS_108_DAY_2023_2023_2024.csv',
 'SURFACE_ASOS_108_DAY_2024_2024_2025.csv',
 'SURFACE_ASOS_108_HR_2022_2022_2023.csv',
 'SURFACE_ASOS_108_HR_2023_2023_2024.csv',
 'SURFACE_ASOS_108_HR_2024_2024_2025.csv']

In [13]:
# 2-2) 중복 데이터 삭제
before = len(power_df)
power_df = power_df.drop_duplicates().copy()
after = len(power_df)

print("중복 제거 전:", before, "제거 후:", after, "제거:", before-after)

중복 제거 전: 9754804 제거 후: 4348560 제거: 5406244


In [14]:
# 2-3) 시간당 → 일당 집계 
# (1) 법정동코드(10자리) 생성: 시군구(5) + 법정동(5)
power_df["SIGUNGU_CD"] = power_df["SIGUNGU_CD"].astype(str).str.zfill(5)
power_df["BJDONG_CD"]  = power_df["BJDONG_CD"].astype(str).str.zfill(5)
power_df["법정동코드"] = power_df["SIGUNGU_CD"] + power_df["BJDONG_CD"]

# (2) 일시(datetime) 만들기
# USE_YM이 YYYYMMDD(8자리)면 "일 단위" 가능
# USE_YM이 YYYYMM(6자리)만 있으면 "일 단위"가 불가능하므로 아래에서 월 단위로 대체 집계됩니다.
power_df["USE_YM"] = power_df["USE_YM"].astype(str)
power_df["USE_HM"] = power_df["USE_HM"].astype(str).str.zfill(4)

ym_len = power_df["USE_YM"].str.len().mode().iloc[0]  # 가장 흔한 길이(6 or 8 등)

if ym_len >= 8:
    # YYYYMMDD + HHMM -> datetime
    power_df["일시"] = pd.to_datetime(power_df["USE_YM"].str[:8] + power_df["USE_HM"],
                                      format="%Y%m%d%H%M", errors="coerce")
    power_df["일자"] = power_df["일시"].dt.date

    # (3) 전력량 숫자 변환 + 일 단위 합계
    power_df["FDRCT_VLD_KWH"] = pd.to_numeric(power_df["FDRCT_VLD_KWH"], errors="coerce")

    power_daily = (
        power_df
        .groupby(["법정동코드", "일자"], as_index=False)["FDRCT_VLD_KWH"]
        .sum()
        .rename(columns={"FDRCT_VLD_KWH": "일사용전력량"})
    )
else:
    # USE_YM이 YYYYMM만 존재하는 케이스: 일 집계 불가 -> 월 집계로 생성
    power_df["FDRCT_VLD_KWH"] = pd.to_numeric(power_df["FDRCT_VLD_KWH"], errors="coerce")

    power_monthly = (
        power_df
        .groupby(["법정동코드", "USE_YM"], as_index=False)["FDRCT_VLD_KWH"]
        .sum()
        .rename(columns={"USE_YM": "년월", "FDRCT_VLD_KWH": "월사용전력량"})
    )

power_daily.head() if ym_len >= 8 else power_monthly.head()

,법정동코드,일자,일사용전력량
0,1111010100,2022-12-04,140153.5620
1,1111010100,2022-12-05,171154.5395
2,1111010100,2022-12-06,152976.6505
3,1111010100,2022-12-08,171442.0635
4,1111010100,2022-12-09,152646.3460


In [17]:
#2-4) 법정동 마스터(dong_clean)와 병합
#일 단위로 성공한 경우(ym_len >= 8)
merged_power = power_daily.merge(dong_clean, on="법정동코드", how="left")

print("병합 후 shape:", merged_power.shape)
print("법정동 매칭 실패:", merged_power["시군구명"].isna().sum())
merged_power.head()

병합 후 shape: (181408, 7)
법정동 매칭 실패: 0


,법정동코드,일자,일사용전력량,시군구코드,법정동코드_하위,시군구명,법정동명_하위
0,1111010100,2022-12-04,140153.5620,11110,10100,서울특별시 종로구,청운동
1,1111010100,2022-12-05,171154.5395,11110,10100,서울특별시 종로구,청운동
2,1111010100,2022-12-06,152976.6505,11110,10100,서울특별시 종로구,청운동
3,1111010100,2022-12-08,171442.0635,11110,10100,서울특별시 종로구,청운동
4,1111010100,2022-12-09,152646.3460,11110,10100,서울특별시 종로구,청운동


In [19]:
# 결과 저장
merged_power.to_csv("data/전력_집계_법정동병합.csv", index=False, encoding="utf-8-sig")

In [20]:
power_df["USE_YM"].astype(str).str.len().value_counts()

8    4348560
Name: USE_YM, dtype: int64

In [28]:
#3단계 날씨 데이터 전처리
#3-1) 일자료(DAY) 먼저 처리 (권장 시작점)
# ① 여러 DAY 파일 로드 & 병합
base_dir = "data/weather_data"

day_files = [
    os.path.join(base_dir, "SURFACE_ASOS_108_DAY_2022_2022_2023.csv"),
    os.path.join(base_dir, "SURFACE_ASOS_108_DAY_2023_2023_2024.csv"),
    os.path.join(base_dir, "SURFACE_ASOS_108_DAY_2024_2024_2025.csv"),
]

day_dfs = []
for f in day_files:
    try:
        df = pd.read_csv(f, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(f, encoding="cp949")
    day_dfs.append(df)

weather_day = pd.concat(day_dfs, ignore_index=True)

print("weather_day shape:", weather_day.shape)
weather_day.head()

weather_day shape: (1096, 59)


,지점,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),강수 계속시간(hr),10분 최다 강수량(mm),10분 최다강수량 시각(hhmi),...,평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr)
0,108,2022-01-01,-4.3,-10.2,710.0,2.3,1544,NaN,NaN,NaN,...,0.9,2.7,6.6,10.1,15.1,17.2,1.3,1.8,0.3,NaN
1,108,2022-01-02,-1.3,-5.2,2356.0,3.0,1551,4.17,NaN,NaN,...,0.8,2.7,6.4,10.0,15.0,17.2,1.1,1.6,NaN,NaN
2,108,2022-01-03,-1.9,-8.0,714.0,2.5,1542,4.00,NaN,NaN,...,0.8,2.5,6.2,9.8,14.9,17.2,1.1,1.6,0.0,NaN
3,108,2022-01-04,-2.5,-5.6,2400.0,1.0,1445,0.92,NaN,NaN,...,0.7,2.5,6.1,9.7,14.8,17.1,1.4,2.0,NaN,NaN
4,108,2022-01-05,-2.8,-7.8,634.0,1.9,1518,NaN,NaN,NaN,...,0.7,2.4,5.9,9.5,14.7,17.1,1.0,1.4,NaN,NaN


In [29]:
#3-1  컬럼 확인
weather_day.columns

Index(['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최저기온 시각(hhmi)', '최고기온(°C)',
       '최고기온 시각(hhmi)', '강수 계속시간(hr)', '10분 최다 강수량(mm)', '10분 최다강수량 시각(hhmi)',
       '1시간 최다강수량(mm)', '1시간 최다 강수량 시각(hhmi)', '일강수량(mm)', '최대 순간 풍속(m/s)',
       '최대 순간 풍속 풍향(16방위)', '최대 순간풍속 시각(hhmi)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)',
       '최대 풍속 시각(hhmi)', '평균 풍속(m/s)', '풍정합(100m)', '평균 이슬점온도(°C)',
       '최소 상대습도(%)', '최소 상대습도 시각(hhmi)', '평균 상대습도(%)', '평균 증기압(hPa)',
       '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최고 해면기압 시각(hhmi)', '최저 해면기압(hPa)',
       '최저 해면기압 시각(hhmi)', '평균 해면기압(hPa)', '가조시간(hr)', '합계 일조시간(hr)',
       '1시간 최다일사 시각(hhmi)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)', '일 최심신적설(cm)',
       '일 최심신적설 시각(hhmi)', '일 최심적설(cm)', '일 최심적설 시각(hhmi)', '합계 3시간 신적설(cm)',
       '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)', '최저 초상온도(°C)',
       '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)', '평균 20cm 지중온도(°C)',
       '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)', '1.0m 지중온도(°C)', '1.5m 지중온도(°C)',
       '3.0m 지중온도(°C)', '5.0m 지중온도(°C

In [ ]:
# 1) 컬럼명 확인(디버깅용)
print("컬럼 목록:", list(weather_day.columns))

# 2) 날짜 후보 컬럼 자동 탐지
date_candidates = []
for c in weather_day.columns:
    cl = str(c).lower()
    if any(k in cl for k in ["tm", "date", "일", "날", "시간", "time", "yyyymmdd", "yyyymm"]):
        date_candidates.append(c)

print("날짜 후보 컬럼:", date_candidates)

# 3) 후보 중에서 실제로 datetime 변환이 가장 잘 되는 컬럼을 선택
best_col = None
best_valid = -1
best_series = None

for c in date_candidates:
    s = pd.to_datetime(weather_day[c], errors="coerce")
    valid = s.notna().sum()
    if valid > best_valid:
        best_valid = valid
        best_col = c
        best_series = s

if best_col is None or best_valid == 0:
    raise ValueError("날짜 컬럼 자동 탐지 실패. 위 '컬럼 목록'과 '날짜 후보 컬럼' 출력값을 확인하세요.")

print(f"선택된 날짜 컬럼: {best_col} (유효 변환 개수: {best_valid})")

# 4) 일자 컬럼 생성
weather_day["일자"] = best_series.dt.date

# 5) 수치형 컬럼 변환(ASOS는 수치가 문자열로 들어오는 경우 많음)
#    '일자' 제외하고 가능한 컬럼은 숫자로 변환 시도
for c in weather_day.columns:
    if c == "일자":
        continue
    if weather_day[c].dtype == "object":
        # 쉼표 제거 후 숫자 변환 시도
        tmp = weather_day[c].astype(str).str.replace(",", "", regex=False)
        # 숫자로 변환 가능한 비율이 높으면 숫자로 확정
        converted = pd.to_numeric(tmp, errors="coerce")
        if converted.notna().sum() >= 0.5 * len(converted):
            weather_day[c] = converted

# 6) 우리가 쓰려는 핵심 컬럼을 "존재하는 것만" 선택
preferred = ["일자", "AVG_TA", "MIN_TA", "MAX_TA", "SUM_RN", "AVG_WS", "AVG_RHM"]
use_cols = [c for c in preferred if c in weather_day.columns]
print("사용 컬럼:", use_cols)

weather_day_clean = weather_day[use_cols].copy()

#3-2 결측치 현황
print("결측치 개수:\n", weather_day_clean.isna().sum())

#  결측치 처리 원칙
# 강수량은 NaN -> 0 (측정 누락과 무강수 구분이 애매할 수 있으나, 일자료에서 실무적으로 0 처리 많이 함)
if "SUM_RN" in weather_day_clean.columns:
    weather_day_clean["SUM_RN"] = weather_day_clean["SUM_RN"].fillna(0)

# 나머지는 날짜 순 정렬 후 forward fill
weather_day_clean = weather_day_clean.sort_values("일자").reset_index(drop=True)
weather_day_clean = weather_day_clean.fillna(method="ffill")

weather_day_clean.head()

컬럼 목록: ['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최저기온 시각(hhmi)', '최고기온(°C)', '최고기온 시각(hhmi)', '강수 계속시간(hr)', '10분 최다 강수량(mm)', '10분 최다강수량 시각(hhmi)', '1시간 최다강수량(mm)', '1시간 최다 강수량 시각(hhmi)', '일강수량(mm)', '최대 순간 풍속(m/s)', '최대 순간 풍속 풍향(16방위)', '최대 순간풍속 시각(hhmi)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)', '최대 풍속 시각(hhmi)', '평균 풍속(m/s)', '풍정합(100m)', '평균 이슬점온도(°C)', '최소 상대습도(%)', '최소 상대습도 시각(hhmi)', '평균 상대습도(%)', '평균 증기압(hPa)', '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최고 해면기압 시각(hhmi)', '최저 해면기압(hPa)', '최저 해면기압 시각(hhmi)', '평균 해면기압(hPa)', '가조시간(hr)', '합계 일조시간(hr)', '1시간 최다일사 시각(hhmi)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)', '일 최심신적설(cm)', '일 최심신적설 시각(hhmi)', '일 최심적설(cm)', '일 최심적설 시각(hhmi)', '합계 3시간 신적설(cm)', '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)', '최저 초상온도(°C)', '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)', '평균 20cm 지중온도(°C)', '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)', '1.0m 지중온도(°C)', '1.5m 지중온도(°C)', '3.0m 지중온도(°C)', '5.0m 지중온도(°C)', '합계 대형증발량(mm)', '합계 소형증발량(mm)', '9-9강수(mm)', '안개 계속시간(hr)']
날짜 후보 컬럼: ['일시', '강수 계속시간(

,일자
0,2022-01-01
1,2022-01-02
2,2022-01-03
3,2022-01-04
4,2022-01-05


In [ ]:
#  시군구코드 부여 (서울: 11000)
weather_day_clean["시군구코드"] = "11000"

In [37]:
# 확인 검증
weather_day_clean.head()
weather_day_clean.columns
weather_day_clean["시군구코드"].value_counts()
# 타입/결측 체크
print(weather_day_clean.shape)
print(weather_day_clean.isna().sum().head(20))
weather_day_clean.head(3)

(1096, 2)
일자       0
시군구코드    0
dtype: int64


,일자,시군구코드
0,2022-01-01,11000
1,2022-01-02,11000
2,2022-01-03,11000


In [ ]:
# 3-3) 전력(merged_power) + 날씨(weather_day_clean) 를 ["일자","시군구코드"] 기준으로 병합
final_df = merged_power.merge(
    weather_day_clean,
    on=["일자", "시군구코드"],
    how="left"
)

print(final_df.shape)
final_df.head()

(181408, 7)


,법정동코드,일자,일사용전력량,시군구코드,법정동코드_하위,시군구명,법정동명_하위
0,1111010100,2022-12-04,140153.5620,11110,10100,서울특별시 종로구,청운동
1,1111010100,2022-12-05,171154.5395,11110,10100,서울특별시 종로구,청운동
2,1111010100,2022-12-06,152976.6505,11110,10100,서울특별시 종로구,청운동
3,1111010100,2022-12-08,171442.0635,11110,10100,서울특별시 종로구,청운동
4,1111010100,2022-12-09,152646.3460,11110,10100,서울특별시 종로구,청운동
